In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [13]:
ecg_feature = pd.read_csv('./ecg_features.csv')
ppg_feature = pd.read_csv('./ppg_features.csv')
eda_feature = pd.read_csv('./eda_features.csv')
feel_feature = pd.read_csv('./feeling_labels.csv')
id_feature = pd.read_csv('./id_labels.csv')

In [14]:
id_feature

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
1,1,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
2,2,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
3,3,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
4,4,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,229,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
230,230,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
231,231,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
232,232,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179


In [15]:
ecg_feature = ecg_feature.drop(['Unnamed: 0'], axis = 1)
ppg_feature = ppg_feature.drop(['Unnamed: 0'], axis = 1)

In [16]:
X_train = pd.concat([ecg_feature, ppg_feature, eda_feature, feel_feature, id_feature], axis = 1).values

In [17]:
label_data = pd.read_csv('./train_label_softmax.csv')
y_train = label_data.values

In [7]:
X_train = np.expand_dims(X_train, axis = 0)

In [18]:
X_train

array([[ 1.2140015e+01,  1.9875168e+01,  0.0000000e+00, ...,
         1.3286816e-02, -2.6401860e-03, -1.0924795e-02],
       [ 1.3152130e+01,  2.1535746e+01,  0.0000000e+00, ...,
         1.3286816e-02, -2.6401860e-03, -1.0924795e-02],
       [ 1.3004108e+01,  2.1283129e+01,  0.0000000e+00, ...,
         1.3286816e-02, -2.6401860e-03, -1.0924795e-02],
       ...,
       [ 1.1715818e+01,  1.9185234e+01,  0.0000000e+00, ...,
         1.5053372e-02,  9.5619885e-03, -2.0179033e-02],
       [ 1.2600520e+01,  2.0633630e+01,  0.0000000e+00, ...,
         1.5053372e-02,  9.5619885e-03, -2.0179033e-02],
       [ 1.0416733e+01,  1.7057709e+01,  0.0000000e+00, ...,
         1.5053372e-02,  9.5619885e-03, -2.0179033e-02]])

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
#model.add(Flatten())
model.fit(X_train, y_train, epochs=500, batch_size=32)

Epoch 1/500
8/8 [==============================] - 0s 5ms/step - loss: 1.8764 - accuracy: 0.1880
Epoch 2/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7951 - accuracy: 0.1239
Epoch 3/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7943 - accuracy: 0.1709
Epoch 4/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7918 - accuracy: 0.2222
Epoch 5/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7914 - accuracy: 0.1923
Epoch 6/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7853 - accuracy: 0.2350
Epoch 7/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7819 - accuracy: 0.1410
Epoch 8/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7843 - accuracy: 0.1923
Epoch 9/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7780 - accuracy: 0.2265
Epoch 10/500
8/8 [==============================] - 0s 5ms/step - loss: 1.7791 - accuracy: 0.2521
Epoch 11/500
8/8 [===========

8/8 [==============================] - 0s 5ms/step - loss: 1.6359 - accuracy: 0.3632
Epoch 85/500
8/8 [==============================] - 0s 5ms/step - loss: 1.6142 - accuracy: 0.3761
Epoch 86/500
8/8 [==============================] - 0s 5ms/step - loss: 1.6167 - accuracy: 0.3846
Epoch 87/500
8/8 [==============================] - 0s 5ms/step - loss: 1.5996 - accuracy: 0.4188
Epoch 88/500
8/8 [==============================] - 0s 5ms/step - loss: 1.5966 - accuracy: 0.4231
Epoch 89/500
8/8 [==============================] - 0s 5ms/step - loss: 1.6087 - accuracy: 0.3333
Epoch 90/500
8/8 [==============================] - 0s 5ms/step - loss: 1.6158 - accuracy: 0.4103
Epoch 91/500
8/8 [==============================] - 0s 5ms/step - loss: 1.5992 - accuracy: 0.3462
Epoch 92/500
8/8 [==============================] - 0s 5ms/step - loss: 1.6107 - accuracy: 0.4103
Epoch 93/500
8/8 [==============================] - 0s 5ms/step - loss: 1.5996 - accuracy: 0.4274
Epoch 94/500
8/8 [===============

8/8 [==============================] - 0s 5ms/step - loss: 1.1478 - accuracy: 0.6709
Epoch 168/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1370 - accuracy: 0.6795
Epoch 169/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1232 - accuracy: 0.7051
Epoch 170/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1013 - accuracy: 0.7051
Epoch 171/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1096 - accuracy: 0.7137
Epoch 172/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1279 - accuracy: 0.6838
Epoch 173/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1123 - accuracy: 0.7137
Epoch 174/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1200 - accuracy: 0.7137
Epoch 175/500
8/8 [==============================] - 0s 5ms/step - loss: 1.1199 - accuracy: 0.7094
Epoch 176/500
8/8 [==============================] - 0s 5ms/step - loss: 1.0771 - accuracy: 0.7179
Epoch 177/500
8/8 [=====

8/8 [==============================] - 0s 5ms/step - loss: 0.8882 - accuracy: 0.8376
Epoch 250/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8990 - accuracy: 0.8034
Epoch 251/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8847 - accuracy: 0.8205
Epoch 252/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8931 - accuracy: 0.8333
Epoch 253/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8487 - accuracy: 0.8419
Epoch 254/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8369 - accuracy: 0.8632
Epoch 255/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8618 - accuracy: 0.8120
Epoch 256/500
8/8 [==============================] - 0s 5ms/step - loss: 0.9477 - accuracy: 0.8120
Epoch 257/500
8/8 [==============================] - 0s 5ms/step - loss: 0.9490 - accuracy: 0.7436
Epoch 258/500
8/8 [==============================] - 0s 5ms/step - loss: 0.8934 - accuracy: 0.8504
Epoch 259/500
8/8 [=====

8/8 [==============================] - 0s 5ms/step - loss: 0.7561 - accuracy: 0.8846
Epoch 332/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7599 - accuracy: 0.8974
Epoch 333/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7451 - accuracy: 0.9103
Epoch 334/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7389 - accuracy: 0.8718
Epoch 335/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7323 - accuracy: 0.8932
Epoch 336/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7376 - accuracy: 0.9145
Epoch 337/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7469 - accuracy: 0.8846
Epoch 338/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7420 - accuracy: 0.8803
Epoch 339/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7441 - accuracy: 0.8889
Epoch 340/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7370 - accuracy: 0.9017
Epoch 341/500
8/8 [=====

8/8 [==============================] - 0s 6ms/step - loss: 0.7230 - accuracy: 0.8932
Epoch 414/500
8/8 [==============================] - 0s 6ms/step - loss: 0.7148 - accuracy: 0.9145
Epoch 415/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7051 - accuracy: 0.9017
Epoch 416/500
8/8 [==============================] - 0s 6ms/step - loss: 0.7102 - accuracy: 0.9145
Epoch 417/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7012 - accuracy: 0.9060
Epoch 418/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7331 - accuracy: 0.8803
Epoch 419/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6944 - accuracy: 0.9188
Epoch 420/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6856 - accuracy: 0.9017
Epoch 421/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6889 - accuracy: 0.8932
Epoch 422/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7025 - accuracy: 0.9103
Epoch 423/500
8/8 [=====

8/8 [==============================] - 0s 5ms/step - loss: 0.7359 - accuracy: 0.8889
Epoch 496/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7142 - accuracy: 0.8718
Epoch 497/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7101 - accuracy: 0.8632
Epoch 498/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7032 - accuracy: 0.8846
Epoch 499/500
8/8 [==============================] - 0s 5ms/step - loss: 0.7010 - accuracy: 0.9017
Epoch 500/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6810 - accuracy: 0.9060


In [ ]:
output_layer = Dense(6, activation='softmax')(x)  

model = Model(inputs=input_layer, outputs=output_layer)